In [59]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_1_1.csv')
df = df.dropna()
df = df[:int(len(df) * 0.7)]
print(round(df.PTRATIO.quantile(0.25), 1))

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_1_2.csv')
sub_df1 = df.loc[(df['yearBuilt'] >= 1991) & (df['yearBuilt'] <= 2000), :]
sub_df2 = df.loc[(df['yearBuilt'] >= 2001) & (df['yearBuilt'] <= 2010), :]
uid1 = sub_df1.loc[df['avgSchoolRating'] <= df['avgSchoolRating'].mean(), 'uid']
uid2 = sub_df2.loc[df['avgSchoolRating'] >= df['avgSchoolRating'].mean(), 'uid']
result = pd.concat([uid1, uid2], axis=0)
print(len(result.unique()))
len1 = df[(df['yearBuilt'] >= 1991) & (df['yearBuilt'] <= 2000) & (df['avgSchoolRating'] <= df['avgSchoolRating'].mean())]['uid']
len2 = df[(df['yearBuilt'] >= 2001) & (df['yearBuilt'] <= 2010) & (df['avgSchoolRating'] >= df['avgSchoolRating'].mean())]['uid']
rs = pd.concat([len1, len2], axis=0)
print(len(rs.unique()))

print(df.isnull().sum().idxmax())

16.4
76
76
numOfPatioAndPorchFeatures


In [34]:
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_2_test.csv')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

key = test['ID']
y_train = train['Attrition_Flag'].map({'Attrited Customer':1, 'Existing Customer':0})
y_test = test['Attrition_Flag'].map({'Attrited Customer':1, 'Existing Customer':0})
x_train = train.drop(['ID', 'Attrition_Flag'], axis=1)
x_test = test.drop(['ID', 'Attrition_Flag'], axis=1)

co = pd.concat([x_train, x_test], axis=0)
obj = ['Income_Category', 'Card_Category']
co = pd.get_dummies(co, columns=obj)
co['Gender'] = co['Gender'].map({'M':True, 'F':False})
mm = MinMaxScaler()
sd = StandardScaler()
num = ['Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy']
co[num] = mm.fit_transform(co[num])
#co[num] = sd.fit_transform(co[num])
x_train = co.iloc[:len(x_train), :]
x_test = co.iloc[len(x_train):, :]

x_t, x_va, y_t, y_va = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
model = RandomForestClassifier(
    n_estimators = 1000,
    max_depth = 5,
    min_samples_leaf = 3,
    min_samples_split = 4,
    random_state = 42,
    class_weight = 'balanced'
)
model.fit(x_t, y_t)
pred = model.predict(x_va)
print('F1 Score:', f1_score(y_va, pred, average='macro'))

result = pd.DataFrame()
result['ID'] = key
result['pred'] = model.predict(x_test)
print(result.isnull().sum().sum(), result.pred.value_counts())
print(result.head(5))
#result.to_csv('result.csv', index=False)

F1 Score: 0.7464285714285714
0 pred
0    113
1     39
Name: count, dtype: int64
     ID  pred
0  6086     0
1  6630     1
2  4292     0
3  7044     0
4  2619     0


In [49]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_3_1.csv')
print(round(df.groupby('department')['hours_worked'].mean(),2), round(df.groupby('department')['hours_worked'].std(),2))
from scipy.stats import ttest_ind
st, p = ttest_ind(df[df['department'] == 'A']['hours_worked'], df[df['department'] == 'B']['hours_worked'])
print(round(st, 2))
print(round(p, 2))
if p < 0.05:
    print('기각')
else:
    print('채택')

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/4_3_2.csv')
print(df.head(2))
from scipy.stats import kruskal
print(round(df.groupby('factory')['quality_score'].mean(),2), round(df.groupby('factory')['quality_score'].std(),2))
g1 = df[df['factory'] == 'A']['quality_score']
g2 = df[df['factory'] == 'B']['quality_score']
g3 = df[df['factory'] == 'C']['quality_score']
st, p = kruskal(g1, g2, g3)
print(round(st, 2))
print(round(p, 2))
if p < 0.05:
    print('기각')
else:
    print('채택')

department
A    42.21
B    43.55
Name: hours_worked, dtype: float64 department
A    5.50
B    4.57
Name: hours_worked, dtype: float64
-1.02
0.31
채택
  factory  quality_score
0       A      87.640523
1       A      74.001572
factory
A    74.43
B    72.10
C    78.66
Name: quality_score, dtype: float64 factory
A    11.00
B     9.14
C     9.65
Name: quality_score, dtype: float64
5.93
0.05
채택
